In [158]:
import random as rdn

In [159]:
class formiga:
    def __init__(self, quantidadeRainhas, evaporacao):
        self.iteracao = 0
        self.custo = quantidadeRainhas
        self.rainhas = quantidadeRainhas
        self.valorEvaporacao = evaporacao
        
        self.caminho = [i for i in range(quantidadeRainhas)] 
        self.matrizFeromonio = [[1 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        self.matrizProbabilidade = [[0 for i in range(quantidadeRainhas)] for j in range(quantidadeRainhas)]
        
    def caminhoInicial(self):
        rdn.shuffle(self.caminho)
    
    def geraMatrizRainhas(self):
        self.matrizRainhas = [[0 for i in range(self.rainhas)] for j in range(self.rainhas)]
        
        for l, c in enumerate(self.caminho):
            self.matrizRainhas[l][c] = 1
    
    def calcularCusto(self):
        dicionario = {}
        for i in self.caminho:
            if i in dicionario:
                dicionario[i] = dicionario[i] + 1
            else:
                dicionario[i] = 1
            
        self.custo = int(self.custoDiagonal() / 2) + self.custoColuna()
    
    def evaporarFeromonio(self):
        for linha in range(self.rainhas):
            for coluna in range(self.rainhas):
                if (coluna != self.caminho[linha]):
                    self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] - self.matrizFeromonio[linha][coluna] * self.valorEvaporacao, 3)
    
    def custoDiagonal(self):
        somaDiagonal = 0 
        
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):

                if self.caminho[i]+(j-i) < len(self.caminho):
                    if self.caminho[i+(j-i)] == self.caminho[i]+(j-i):
                        somaDiagonal += 1
                if self.caminho[i]-(j-i) >= 0:
                    if self.caminho[i+(j-i)] == self.caminho[i]-(j-i):
                        somaDiagonal += 1

            for j in range(0,i):
                if self.caminho[i]+(j+1) < len(self.caminho):
                    if self.caminho[i-(j+1)] == self.caminho[i]+(j+1):
                        somaDiagonal += 1
                if self.caminho[i]-(j+1) >= 0:
                    if self.caminho[i-(j+1)] == self.caminho[i]-(j+1):
                        somaDiagonal += 1
        
        return somaDiagonal
    
    def custoColuna(self):
        custo = 0
        for i in range(len(self.caminho)):
            for j in range(i+1, len(self.caminho)):
                if self.caminho[i] == self.caminho[j]:
                    custo += 1
        return custo
    
    def calcularMatrizFeromonio(self):
        self.calcularCusto()
        
        if (self.custo == 0):
            return self.caminho

        feromonio = round(1 / self.custo, 3)
        
        self.evaporarFeromonio()
        
        for linha, coluna in enumerate(self.caminho):
            self.matrizFeromonio[linha][coluna] = round(self.matrizFeromonio[linha][coluna] + feromonio, 3)
                
    def calcularMatrizProbabilidade(self):
        somas = [0 for i in range(self.rainhas)]
            
        for linha in self.matrizFeromonio:    
            for indice, coluna in enumerate(linha):
                somas[indice] += coluna

        for indiceLinha, linha in enumerate(self.matrizFeromonio):  
            for indiceColuna, coluna in enumerate(linha):
                self.matrizProbabilidade[indiceLinha][indiceColuna] = round(coluna / somas[indiceColuna], 3)
                              
    def novoCaminhoFormiga(self):
        i = 0
        for line in self.matrizProbabilidade:
            roleta = [0 for n in line]
            somaPeso = sum(line)
            sorteio = rdn.random()
            
            prob = [x*100/somaPeso for x in line]
            
            roleta[0] = (0, prob[0])
            for x in range(1, len(prob)):
                roleta[x] = (roleta[x-1][1], roleta[x-1][1] + prob[x])

            self.caminho[i] = self.escolherPosicaoArbitraria(roleta, sorteio)
            i += 1
    
    def escolherPosicaoArbitraria(self, roleta, sorteio):
        sorteio *= 100
        posicao = 0
        for minimo, maximo in roleta:
            
            if sorteio >= minimo and sorteio < maximo:
                return posicao
            
            posicao += 1
        
        return posicao
    
    def debug(self):
        print('Iteracao', self.iteracao)
        print('Custo', self.custo)
        
        print('Rainhas')
        self.printMatriz(self.matrizRainhas)
        
        print('Feromonios')
        self.printMatriz(self.matrizFeromonio)
        
        print('Probabilidade')
        self.printMatriz(self.matrizProbabilidade)
                
    def printMatriz(self, matriz):
        for linha in matriz:
            for coluna in linha:
                print(coluna, end='  ')
            print()
        print()
    
    def encontrarSolucao(self):
        self.caminhoInicial()
        
        while (self.custo > 0):
            self.geraMatrizRainhas()
            self.calcularMatrizFeromonio()
            
            self.debug()
            
            self.calcularMatrizProbabilidade()
            self.novoCaminhoFormiga()
            
            self.iteracao += 1
            
        if self.custo == 0:     
            return self

In [160]:
def main(rainhas, valorEvaporacao):
    formigaSolucao = formiga(rainhas, valorEvaporacao)
    return formigaSolucao.encontrarSolucao()

In [163]:
formigaSolucao = main(40, 0.001)
formigaSolucao.matrizRainhas

Iteracao 0
Custo 2
Rainhas
1  0  0  0  0  
0  0  1  0  0  
0  0  0  1  0  
0  1  0  0  0  
0  0  0  0  1  

Feromonios
1.5  0.999  0.999  0.999  0.999  
0.999  0.999  1.5  0.999  0.999  
0.999  0.999  0.999  1.5  0.999  
0.999  1.5  0.999  0.999  0.999  
0.999  0.999  0.999  0.999  1.5  

Probabilidade
0  0  0  0  0  
0  0  0  0  0  
0  0  0  0  0  
0  0  0  0  0  
0  0  0  0  0  

Iteracao 1
Custo 2
Rainhas
0  0  0  0  1  
0  0  1  0  0  
1  0  0  0  0  
0  0  1  0  0  
0  0  0  1  0  

Feromonios
1.498  0.998  0.998  0.998  1.499  
0.998  0.998  2.0  0.998  0.998  
1.499  0.998  0.998  1.498  0.998  
0.998  1.498  1.499  0.998  0.998  
0.998  0.998  0.998  1.499  1.498  

Probabilidade
0.273  0.182  0.182  0.182  0.182  
0.182  0.182  0.273  0.182  0.182  
0.182  0.182  0.182  0.273  0.182  
0.182  0.273  0.182  0.182  0.182  
0.182  0.182  0.182  0.182  0.273  

Iteracao 2
Custo 4
Rainhas
0  0  0  1  0  
0  0  0  0  1  
1  0  0  0  0  
0  0  1  0  0  
0  1  0  0  0  

Feromonios
1.4

1.561  2.966  2.662  1.894  5.368  
1.967  1.181  0.98  3.336  7.006  
6.46  3.139  2.252  1.382  1.231  
1.231  1.931  1.481  8.167  1.667  

Probabilidade
0.211  0.325  0.185  0.082  0.213  
0.11  0.244  0.294  0.126  0.259  
0.138  0.097  0.108  0.222  0.356  
0.454  0.175  0.249  0.092  0.073  
0.087  0.159  0.164  0.477  0.099  

Iteracao 20
Custo 4
Rainhas
1  0  0  0  0  
0  0  1  0  0  
0  1  0  0  0  
0  1  0  0  0  
0  1  0  0  0  

Feromonios
4.262  3.959  1.667  1.23  3.575  
1.559  2.963  2.912  1.892  5.363  
1.965  1.431  0.979  3.333  6.999  
6.454  3.389  2.25  1.381  1.23  
1.23  2.181  1.48  8.159  1.665  

Probabilidade
0.263  0.301  0.185  0.077  0.19  
0.102  0.225  0.294  0.118  0.285  
0.129  0.09  0.108  0.208  0.372  
0.424  0.238  0.249  0.086  0.065  
0.081  0.147  0.164  0.51  0.088  

Iteracao 21
Custo 4
Rainhas
0  0  0  1  0  
0  0  1  0  0  
0  0  0  0  1  
1  0  0  0  0  
0  0  0  1  0  

Feromonios
4.258  3.955  1.665  1.48  3.571  
1.557  2.96  3.162  

[[0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0]]